# Introduction

My aim of this project is to analyse the CIA Factbook Data Using SQL. I will be exploring the data to begin with then start to answer a variation of questions to gather information. 

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

## Overview of the Data

The code below shows the different tables in the SQLite Database. 

I will be concentrating on the `facts` table. 

In [2]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


[('table', 'sqlite_sequence', 'sqlite_sequence', 3, 'CREATE TABLE sqlite_sequence(name,seq)'),
 ('table', 'facts', 'facts', 47, 'CREATE TABLE "facts" ("id" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, "code" varchar(255) NOT NULL, "name" varchar(255) NOT NULL, "area" integer, "a ... (4 characters truncated) ... land" integer, "area_water" integer, "population" integer, "population_growth" float, "birth_rate" float, "death_rate" float, "migration_rate" float)')]

The code below shows the top 5 rows of the data. This is a good start to show us what we are dealing with. 

In [3]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

Done.


[(1, 'af', 'Afghanistan', 652230, 652230, 0, 32564342, 2.32, 38.57, 13.89, 1.51),
 (2, 'al', 'Albania', 28748, 27398, 1350, 3029278, 0.3, 12.92, 6.58, 3.3),
 (3, 'ag', 'Algeria', 2381741, 2381741, 0, 39542166, 1.84, 23.67, 4.31, 0.92),
 (4, 'an', 'Andorra', 468, 468, 0, 85580, 0.12, 8.13, 6.96, 0.0),
 (5, 'ao', 'Angola', 1246700, 1246700, 0, 19625353, 2.78, 38.78, 11.49, 0.46)]

Here are the descriptions for some of the columns:

* `name` - The name of the country.
* `area` - The total land and sea area of the country.
* `population` - The country's population.
* `population_growth` - The country's population growth as a percentage.
* `birth_rate` - The country's birth rate, or the number of births a year per 1,000 people.
* `death_rate` - The country's death rate, or the number of death a year per 1,000 people.
* `area` - The country's total area (both land and water).
* `area_land` - The country's land area in square kilometers.
* `area_water` - The country's waterarea in square kilometers.

# Summary of Statistics

The code below displays the minimum and maximum for columns `population` and `population_growth`.

In [4]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) max_pop_growth 
  FROM facts;

Done.


[(0, 7256490011, 0.0, 4.02)]

The results of the code shows us that there is a country/ies with a population of 0 and a max of `7256490011`.

In the next code cells, I will investigate further what these countries are. 

## Finding Outliers

In [5]:
%%sql
SELECT *
  FROM facts
 WHERE population == 0;

Done.


[(250, 'ay', 'Antarctica', None, 280000, None, 0, None, None, None, None)]

It seems that the culprit is Antarctica. This explains the population of 0. 

In [6]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MAX(population)
                        FROM facts
                     );

Done.


[(261, 'xx', 'World', None, None, None, 7256490011, 1.08, 18.6, 7.8, None)]

In addition, to finding that the population of 0 was Antartica, the above results show that the data includes the `World` as a coutnry. 

I will repeat the summary of statistics but excluding the row for the World and Antartica. 

In [7]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) max_pop_growth 
  FROM facts
 WHERE name != 'World'
   AND name != 'Antarctica';

Done.


[(48, 1367485388, 0.0, 4.02)]

Now that the World and Antartica has been removed. We can see that there is a country with a population as low as `48` and as high as `1.4` Billion. 

This makes me curious in what these countries are. 

In [8]:
%%sql
SELECT Name, population
  FROM facts
 WHERE name != 'World'
   AND name != 'European Union'
 ORDER BY population DESC
 LIMIT 3
;


Done.


[('China', 1367485388), ('India', 1251695584), ('United States', 321368864)]

In [9]:
%%sql
SELECT Name, population
  FROM facts
 WHERE name != 'Antarctica'
   AND population != 'NONE'
 ORDER BY population 
 LIMIT 3
;

Done.


[('Pitcairn Islands', 48),
 ('Cocos (Keeling) Islands', 596),
 ('Holy See (Vatican City)', 842)]

# Exploring Average Population and Area¶

Let's explore density. Density depends on the population and the country's area. Let's look at the average values for these two columns.

I will continue to exlcude the World, so it doesn't effect our results. 

In [12]:
%%sql
SELECT AVG(population) AS 'Average Population',
       AVG(area) AS ' Average Area'
  FROM facts
 WHERE name != 'World';

Done.


[(32242666.56846473, 555093.546184739)]

We see that the average population is around 32 million and the average area is 555 thousand square kilometers.

# Finding Densely Populated Countries

To finish, we'll build on the query above to find countries that are densely populated. We'll identify countries that have:

Above average values for population.
Below average values for area.

In [11]:
%%sql
SELECT *
  FROM facts
 WHERE population > (SELECT AVG(population)
                       FROM facts
                     WHERE name != 'World'
                    )
   AND area < (SELECT AVG(area)
                 FROM facts
               WHERE name != 'World'
              );

Done.


[(14, 'bg', 'Bangladesh', 148460, 130170, 18290, 168957745, 1.6, 21.14, 5.61, 0.46),
 (65, 'gm', 'Germany', 357022, 348672, 8350, 80854408, 0.17, 8.47, 11.42, 1.24),
 (80, 'iz', 'Iraq', 438317, 437367, 950, 37056169, 2.93, 31.45, 3.77, 1.62),
 (83, 'it', 'Italy', 301340, 294140, 7200, 61855120, 0.27, 8.74, 10.19, 4.1),
 (85, 'ja', 'Japan', 377915, 364485, 13430, 126919659, 0.16, 7.93, 9.51, 0.0),
 (91, 'ks', 'Korea, South', 99720, 96920, 2800, 49115196, 0.14, 8.19, 6.75, 0.0),
 (120, 'mo', 'Morocco', 446550, 446300, 250, 33322699, 1.0, 18.2, 4.81, 3.36),
 (138, 'rp', 'Philippines', 300000, 298170, 1830, 100998376, 1.61, 24.27, 6.11, 2.09),
 (139, 'pl', 'Poland', 312685, 304255, 8430, 38562189, 0.09, 9.74, 10.19, 0.46),
 (163, 'sp', 'Spain', 505370, 498980, 6390, 48146134, 0.89, 9.64, 9.04, 8.31),
 (173, 'th', 'Thailand', 513120, 510890, 2230, 67976405, 0.34, 11.19, 7.8, 0.0),
 (182, 'ug', 'Uganda', 241038, 197100, 43938, 37101745, 3.24, 43.79, 10.69, 0.74),
 (185, 'uk', 'United Kingdom', 243610, 241930, 1680, 64088222, 0.54, 12.17, 9.35, 2.54),
 (192, 'vm', 'Vietnam', 331210, 310070, 21140, 94348835, 0.97, 15.96, 5.93, 0.3)]

It is generally known that many of these countries are known to be dense in population and area. So I am confident that the above results are correct. 

